# 1D Helmholtz Multilevel Development: Repetitive; Local Coarsening Derivation
* Constant $k$.
* Discretization: 5-point (4th order).
* Kaczmarz relaxation.
* Fixed-domain problem; repetitive, so we sample windows from a test vector.

In [16]:
%run /Users/olivne/helmholtz/src/helmholtz/startup.ipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Fixed seed for reproducible results.
np.random.seed(1)

# Domain size.
n = 96
# Scaled wave number. Fit so lam_min = 0 (integer # periods in domain).
kh = helmholtz.analysis.ideal.find_singular_kh("5-point", n)[0]

repetitive = True

# Number of test vectors.
num_examples = 3
threshold = 0.1

# Boottstrapping parameters.
interpolation_method = "ls"
neighborhood = "extended" #"aggregate" # "extended"
num_test_examples = 5
leeway_factor = 1.3

_LOGGER.info("kh {}".format(kh))

NameError: name 'np' is not defined

## Level 0->1 Coarsening

In [ ]:
# Create fine-level matrix.
a = hm.linalg.helmholtz_1d_discrete_operator(kh, "5-point", n) #, bc="bloch")
# Use default Kacmzarz for kh != 0.
level = hm.setup.hierarchy.create_finest_level(a, relaxer=hm.solve.relax.GsRelaxer(a) if kh == 0 else None)

# Initialize hierarchy to 1-level.
finest = 0
multilevel = hm.setup.hierarchy.multilevel.Multilevel.create(level)

# TV and TV residual history.
x_log = []
r_log = []

### Relaxation

In [ ]:
method_info = hm.solve.smoothing.check_relax_cycle_shrinkage(
    multilevel, num_levels=1, leeway_factor=leeway_factor, slow_conv_factor=0.95)

In [ ]:
# Generate relaxed vectors.
level = multilevel[0]
_LOGGER.info("Generating {} TVs with {} sweeps".format(x.shape[1], num_sweeps))
x = hm.setup.auto_setup.get_test_matrix(a, num_sweeps, num_examples=num_examples)
_LOGGER.info("RER {:.3f}".format(norm(level.a.dot(x)) / norm(x)))
x_log.append(x)

### Coarsening: Fixed (4/2)

In [ ]:
# Coarsening. Force 2 coarse vars per aggregate so we can test alignment.
aggregate_size = 4
num_components = 2

# Optimize aggregate_size, num_components using mock cycle rates.
# coarsener = hm.setup.coarsening_uniform.UniformCoarsener(
#     level, x, num_sweeps, repetitive=repetitive)
# info = coarsener.get_coarsening_info(1, fmt="dataframe")
# r, aggregate_size, nc, cr, mean_energy_error, mock_conv, mock_work, mock_efficiency = \
#     coarsener.get_optimal_coarsening(1)
# _LOGGER.info("R {} a {} nc {} cr {:.2f} mean_energy_error {:.4f}; mock cycle num_sweeps {} conv {:.2f} "
#              "eff {:.2f}".format(
#     r.shape, aggregate_size, nc, cr, mean_energy_error, num_sweeps, mock_conv, mock_efficiency))
# coarsener.get_coarsening_info(1, fmt="dataframe")

In [ ]:
r = create_coarsening(x, aggregate_size, num_components)
R = r.tile(level.size // aggregate_size)
xc = R.dot(x)
display(pd.DataFrame(R[:5,:10].todense()))
hm.repetitive.locality.plot_coarsening(R, x)

### Local Mock Cycle (LMC) Rate
The mock cycle rate is calculated on a domain of size $4 a$, $a$ = aggregate size.

In [ ]:
# Calculate local mock cycle convergence rate on a domain of size 4 * aggregate_size.
m = 4
nu_values = np.arange(1, num_sweeps + 1)
mock_conv = hm.repetitive.locality.mock_conv_factor_for_domain_size(kh, r, aggregate_size, m * aggregate_size, nu_values)
_LOGGER.info("Mock cycle conv |domain|={} {}".format(
    m * aggregate_size, np.array2string(mock_conv, precision=3)))

### Interpolation
Using $P = R^T$ to start.

In [ ]:
# caliber = 2
# interpolation_method = "svd"
# neighborhood = "extended"

# p = create_interpolation(
#     x_level, level.a, r, interpolation_method, aggregate_size=aggregate_size, nc=num_components, 
#     neighborhood=neighborhood, repetitive=repetitive, target_error=target_error,
#     caliber=caliber)

# for title, x_set in ((("all", x),) if repetitive else (("fit", x_fit), ("test", x_test))):
#     error = norm(x_set - p.dot(r.dot(x_set)), axis=0) / norm(x_set, axis=0)
#     error_a = norm(level.a.dot(x_set - p.dot(r.dot(x_set))), axis=0) / norm(x_set, axis=0)
#     _LOGGER.info(
#         "{:<4s} set size {:<2d} P L2 error mean {:.2f} max {:.2f} A error mean {:.2f} max {:.2f}".format(
#             title, len(error), np.mean(error), np.max(error), np.mean(error_a), np.max(error_a)))

In [ ]:
# Initial guess for interpoation: P = R^T on a single aggregate.
p = r.tile(1).transpose()
print(p.todense())

### Local Two-level Cycle (L2C) Rate

In [ ]:
# nu = 2
# local_multilevel = hm.repetitive.locality.create_two_level_hierarchy(kh, m * aggregate_size, r.asarray(), p, aggregate_size)
# y, _ = hm.repetitive.locality.two_level_conv_factor(local_multilevel, nu, print_frequency=1)

# # Asymptotic vector.
# plt.title("Asymptotically Slowest Vector in Two-level Cycle({},0)".format(nu))
# e = y - local_multilevel[1].p.dot(local_multilevel[1].r.dot(y))
# plt.plot(y, label="x");
# plt.plot(e, label="x - PRx");
# plt.grid(True);
# plt.legend();

In [ ]:
two_level_conv = np.array([
    two_level_conv_factor(hm.repetitive.locality.create_two_level_hierarchy(
        kh, m * aggregate_size, r.asarray(), p, aggregate_size), nu)[1]
     for nu in nu_values])

conv = pd.DataFrame(np.array((mock_conv, two_level_conv)).transpose(), 
                    index=nu_values, columns=("Mock", "Two-level")).transpose()
display(conv)

# Compare the L2 projection norm and the A*A'-orthogonal projection norms, which gives an indication
# that the mock cycle rates are guaranteed to be good predictors of the two-level rates [Rob & James].
R = r.tile(n // aggregate_size)
_LOGGER.info("L2 projection norm {:.2f} A*A' projection norm {:.2f}".
             format(norm(R.todense(), ord=2), norm((R.dot(multilevel[0].a)).todense(), ord=2)))

This intepolation is good up to $\nu = 2$, conv $\approx 0.5$.

<!-- ### Build Coarse Level: Two-level Bootstrap Cycle -->

In [ ]:
# max_levels = 2
# num_bootstrap_steps = 1

# # Bootstrap with an increasingly deeper hierarchy (add one level at a time).
# num_levels = 2
# _LOGGER.info("bootstrap on grid size {} with {} levels".format(x.shape[0], max_levels))
# _LOGGER.info("-" * 80)
# for i in range(num_bootstrap_steps):
#     _LOGGER.info("Bootstrap step {}/{}".format(i + 1, num_bootstrap_steps))
#     # Set relax_conv_factor to a high value so that we never append a bootstrap vector to the TV set.
#     x, multilevel = hm.setup.auto_setup.bootstap(
#         x, multilevel, num_levels, 2.0,
#         num_sweeps=num_sweeps, interpolation_method=interpolation_method, 
#         neighborhood=neighborhood, repetitive=repetitive, target_error=0.1)
#     x_log.append(x)
#     r_log.append(multilevel[1].r)
#     _LOGGER.info("RER {:.6f}".format(norm(a.dot(x)) / norm(x)))
#     _LOGGER.info("-" * 80)

### Two-level Hierarchy

In [ ]:
multilevel = hm.hierarchy.multilevel.Multilevel.create(level)
l = 1
r_csr = r.tile(n // aggregate_size)
p_csr = r_csr.transpose()
level1 = hm.setup.hierarchy.create_coarse_level(level.a, level.b, r_csr, p_csr)
_LOGGER.info("Level {} size {}".format(l, level1.size))
multilevel.add(level1)

#### Operators

In [ ]:
sz = 8
display(pd.DataFrame(multilevel[0].a.todense()[:sz,:sz]))
display(pd.DataFrame(multilevel[1].r[:sz,:sz].todense()))
display(pd.DataFrame(multilevel[1].p[:sz,:sz].todense()))
display(pd.DataFrame(multilevel[1].a[:sz,:sz].todense()))

#### Level 1

In [ ]:
pd.DataFrame(multilevel[1].a[:10,:10].todense())

In [ ]:
pd.DataFrame(multilevel[1].p[:10, :10].todense())

In [ ]:
pd.DataFrame(multilevel[1].r[:10,:10].todense())

### Coarse-level Relaxation Shrinkage

In [ ]:
work = 1
l = 1
level = multilevel[l]
_LOGGER.info("Relax at level {} size {}".format(l, level.size))
b = np.zeros((level.size, num_examples))

factor, num_coarse_sweeps, residual, conv, rer, relax_conv_factor = \
    hm.solve.smoothing.shrinkage_factor(
        level.operator, lambda x, b: level.relax(x, b), (level.size, ), 
        print_frequency=1, max_sweeps=20, slow_conv_factor=0.95, leeway_factor=leeway_factor)
_LOGGER.info("Relax conv {:.2f} shrinkage {:.2f} PODR RER {:.2f} after {} sweeps. Work {:.1f} eff {:.2f}".format(
    relax_conv_factor, factor, np.mean(rer[num_coarse_sweeps]), num_coarse_sweeps, work, np.mean(residual[num_coarse_sweeps] / residual[0]) ** (1/(num_coarse_sweeps * work))))

title = "Relax"
color = "blue"
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
hm.solve.smoothing.plot_diminishing_returns_point(factor, num_sweeps, conv, ax, title=title, color=color)
print("{:<5s} conv {:.2f} shrinkage {:.2f} PODR RER {:.2f} after {:>2d} sweeps. Work {:>2d} efficiency {:.2f}".format(
    title, conv_factor, factor, np.mean(rer[num_coarse_sweeps]), num_coarse_sweeps, work, np.mean(residual[num_coarse_sweeps] / residual[0]) ** (1/(num_coarse_sweeps * work))))
ax.set_title(r"$kh = {:.2f}$".format(kh))
ax.legend();

### Relaxation Cycle Shrinkage
We compare relaxation cycle with $\nu_1=2, \nu_2=2, \nu_{coarse}=20$ with the resulting $P$ and $R$ from the bootstrap step, with Kaczmarz relaxation.

In [ ]:
slow_conv_factor = 0.9
nu_pre, nu_post, nu_coarsest = num_sweeps, 0, max(3, num_coarse_sweeps)
method_info = hm.solve.smoothing.check_relax_cycle_shrinkage(
    multilevel, nu_pre=nu_pre, nu_post=nu_post, nu_coarsest=nu_coarsest, leeway_factor=leeway_factor, slow_conv_factor=slow_conv_factor)
num_mini_cycles = method_info["Mini-cycle"][2].shape[0]
_LOGGER.info("Two-level ({}, {}; {}) relaxation cycle is slow after {} steps".format(
    nu_pre, nu_post, nu_coarsest, num_mini_cycles))

### Improve Test Vectors by Relaxation Cycles

In [ ]:
def relax_cycle(x):
    return hm.solve.relax_cycle.relax_cycle(multilevel, 1.0, nu_pre, nu_post, nu_coarsest).run(x)

level = multilevel[0]
_LOGGER.info("RER {:.3f}".format(norm(level.a.dot(x)) / norm(x)))
_LOGGER.info("Improving vectors by {} relaxation cycles".format(num_mini_cycles))
x, _ = hm.solve.run.run_iterative_method(level.operator, relax_cycle, x, num_mini_cycles)
_LOGGER.info("RER {:.3f}".format(norm(level.a.dot(x)) / norm(x)))
x_log.append(x)

In [ ]:
aggregate_size = 4
threshold = 0.1
fig, axs = plt.subplots(len(x_log), 3, figsize=(16, 3 * len(x_log)))
for row, x in enumerate(x_log):
    x_aggregate_t = np.concatenate(
        tuple(hm.linalg.get_window(x, offset, aggregate_size)
              for offset in range(max((4 * aggregate_size) // x.shape[1], 1))), axis=1).transpose()    
#     start, end = 0, aggregate_size
#     x_aggregate_t = x[start:end].transpose()
    r, s = hrc.create_coarsening(x_aggregate_t, threshold)
    r = r.asarray()

    # Relaxed vectors.
    ax = axs[row, 0]
    for i in range(3):
        ax.plot(x[:, i]);
    ax.grid(True)
    ax.set_title(r"Test Vectors at Step {}".format(row))

    ax = axs[row, 1]
    # R should be real-valued, but cast just in case.
    for i, ri in enumerate(np.real(r)):
        ax.plot(ri)
    ax.set_title(r"PC Agg Size {} $n_c$ {}".format(r.shape[1], r.shape[0]))
    ax.set_ylabel(r"$R$ rows")
    ax.grid(True);

    # Singular values, normalized to sigma_max = 1.
    ax = axs[row, 2]
    ax.plot(s / s[0], "rx")
    ax.set_title("Singular Values")
    ax.set_xlabel(r"$k$")
    ax.set_ylabel(r"$\sigma_k$")
    ax.grid(True);
    
    print("Step {:2d}".format(row), "s", s / s[0], "Energy error", (1 - np.cumsum(s ** 2) / sum(s ** 2)) ** 0.5)

## Alignment

In [ ]:
level = multilevel[0]
coarse_level = multilevel[1]
xc = coarse_level.coarsen(x)

hm.setup.alignment.calculate_local_repetitive_rotation_angle(4 * aggregate_size, num_components, xc);

# Local alignment.
_LOGGER.info("Local alignment")
phi = hm.setup.alignment.calculate_local_rotation_angles(level.size, aggregate_size, num_components, xc)

# Global alignment.
_LOGGER.info("Global alignment")
f, tmin = hm.setup.alignment.optimal_rotation_angle(xc)

# Plot f(theta).
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

ax = axs[0]
hm.setup.alignment.plot_min_functions(n, aggregate_size, num_components, xc, ax)
ax.set_title("Local Alignment");

ax = axs[1]
t = np.linspace(0, 2 * np.pi, 100)
ax.plot(t / np.pi, np.array([f(theta) for theta in t]))
ax.grid(True);
ax.set_xlabel(r"$\theta / \pi$")
ax.set_ylabel(r"$f(\theta)$");
ax.set_title("Global Alignment");

In [ ]:
# pd.DataFrame(multilevel[1].a.sum(axis=1))

<!-- ### Solving $A x = b$ (Periodic Fixed-Size Domain Problem)
That is, solving on a periodic fixed d
omain. $b$ is a random periodic vector. We start from random $x$. Solving exactly on the coarsest level works fine despite the indefiniteness since the matrix is not (even nearly) singular. -->

In [ ]:
# level = multilevel.finest_level
# # Test two-level cycle convergence for A*x=0 and  A*x=b with random b.
# for title, b in (("0", np.zeros((a0.shape[0], ))), ("b", np.random.random((a0.shape[0], )))):
#     logger.info("Ax={}".format(title))
#     two_level_cycle = lambda y: hm.solve.solve_cycle.solve_cycle(multilevel, 1.0, 2, 1, nu_coarsest=-1, debug=False, rhs=b).run(y)
#     residual = lambda x: b - multilevel[0].operator(x)
#     x, conv_factor = hm.solve.run.run_iterative_method(residual, two_level_cycle, np.random.random((a0.shape[0], )), 20, print_frequency=1)

In [ ]:
# # Asymptotic vector.
# e = x - multilevel[1].p.dot(multilevel[1].r.dot(x))
# plt.plot(x);
# plt.plot(e);

In [ ]:
# # L2 interpolation error
# logger.info("|x-P*R*x|     {:.2e}".format(hm.linalg.scaled_norm(e)))
# logger.info("|x|           {:.2e}".format(hm.linalg.scaled_norm(x)))

# # Residual norm interpolation error
# logger.info("|A*(x-P*R*x)| {:.2e}".format(hm.linalg.scaled_norm(multilevel[0].a.dot(e))))
# logger.info("|Ax|          {:.2e}".format(hm.linalg.scaled_norm(multilevel[0].a.dot(x))))

In [ ]:
# nu_values = np.arange(1, 7, dtype=int)
# r = multilevel[1].r
# two_level_cycle = lambda y: hm.solve.solve_cycle.solve_cycle(multilevel, 1.0, 1, 0, nu_coarsest=-1, debug=False, rhs=b).run(y)
# residual = lambda x: b - multilevel[0].operator(x)
# b = np.random.random((a0.shape[0], ))
# mock_conv_factor = np.array([
#     hm.setup.auto_setup.mock_cycle_conv_factor(multilevel.finest_level, r, nu) 
#     for nu in nu_values])
# two_level_cycle = np.array([
#     hm.solve.run.run_iterative_method(
#         residual, 
#         lambda x: hm.solve.solve_cycle.solve_cycle(multilevel, 1.0, nu, 0, nu_coarsest=-1, rhs=b).run(x), 
#         np.random.random((multilevel.finest_level.size, )), 20)[1]
#     for nu in nu_values])

# for nu, mock, two_level in zip(nu_values, mock_conv_factor, two_level_cycle):
#     print("V({}, {}) conv factor {:.3f} mock cycle {:.3f}".format(nu, 0, two_level, mock))

<!-- For some reason, 1 relaxation per cycle is more efficient than $2-4$ per cycle! Note that we are solving $Ax=b$, not $Ax=0$. -->

## Spectra of Different Levels

In [ ]:
# # Calculate eigenpairs at all levels.
# vl = []
# laml = []
# for l, level in enumerate(multilevel):
#     a = level.a
#     lam, v = eig(a.todense())
#     lam = np.real(lam)
#     ind = np.argsort(np.abs(lam))
#     lam = lam[ind]
#     v = v[:, ind]
#     vl.append(v)
#     laml.append(lam)
#     print(l, "lam", lam[:13])
    
# # Interpolate eigenvectors at all levels to the finest level.
# num_levels = len(multilevel)
# vl_finest = []
# for l in range(num_levels):
#     v = vl[l]
#     for k in range(l, 0, -1):
#         v = multilevel[k].p.dot(v)
#     vl_finest.append(v)

In [ ]:
# num_ev = 8
# num_levels = len(multilevel)
# fig, axs = plt.subplots(num_ev, num_levels, figsize=(16, 16))

# for col, ax in enumerate(axs[0]):
#     ax.set_title("Level {}".format(col))

# for i in range(num_ev):
#     for l in range(num_levels):
#         ax = axs[i, l]
#         ax.plot(np.real(vl[l][:, i]), label="$\lambda_i = {:.3f}$".format(laml[l][i]))
#         ax.legend(loc="upper right")
#         ax.grid(True);